In [1]:
import xarray as xr
import numpy as np
from matplotlib.patches import Ellipse
from scipy.integrate import simps

In [2]:
def confidence_ellipse(x, y, n_std=0.1, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the square root of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)
    print('Ellipse major axis: ' + str(mean_x+scale_x-mean_x+scale_x) + ' K')
    print('Min x value of major axis: ' + str(mean_x-scale_x) + ' K')
    print('Max x value of major axis: ' + str(mean_x+scale_x) + ' K')
    min_majoraxis = mean_x - scale_x
    max_majoraxis = mean_x + scale_x

    # calculating the standard deviation of y ...
    #scale_y = np.sqrt(cov[1, 1]) * n_std
    #mean_y = np.mean(y)

    #transf = transforms.Affine2D() \
    #    .rotate_deg(45) \
    #    .scale(scale_x, scale_y) \
    #    .translate(mean_x, mean_y)

    #ellipse.set_transform(transf + ax.transData)
    return min_majoraxis, max_majoraxis  #ax.add_patch(ellipse), 

In [3]:
# Perform the quadratic qi(T) fitting for the longwave heating scaling
def qiT_fitting( infile, k ):
    bd = '/xdisk/sylvia/traj_output/'
    file = xr.open_dataset( bd + infile )
    T = np.array(file['T'][:k]).ravel()
    IWC = np.array(file['qi'][:k]).ravel()
    
    # Filter these values for non-nan, non-negligible ice water contents
    i = np.argwhere( ~np.isnan(IWC) )
    T = T[i[:,0]]
    IWC = IWC[i[:,0]] * 10**6
    j = np.argwhere( (IWC > 10**(-5)) )
    T = T[j[:,0]]
    IWC = IWC[j[:,0]]
    
    # Find the minimum and maximum temperature from the 0.5*sigma ellipse
    minT, maxT = confidence_ellipse( T, np.log10(IWC) )
    
    # Then filter values for temperatures in teh 0.5*sigma ellipse
    j = np.argwhere( (T > minT) & (T < maxT) )
    T = T[j[:,0]]
    IWC = IWC[j[:,0]]
    print( 'Sample size: ' + str(len(IWC)) )
    
    # Curve fit
    z = np.polyfit( T, IWC, 2 )
    return z, minT, maxT

In [4]:
# Lifecycle 3 fitting
z3, minT3, maxT3 = qiT_fitting( 'CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter.nc', 12000 )
print(z3)
print(minT3,maxT3)

# Lifecycle 3 IWP calculation
T_arr = np.linspace( minT3, maxT3, 100 )
qi_arr = z3[0]*T_arr**2 + z3[1]*T_arr + z3[2]
iwp3 = simps(qi_arr*10**(-6)/0.0055, T_arr ) # here in kg m-2

print('IWP [g m-2] - lifecycle 3: ' + str(iwp3*10**3) )

Ellipse major axis: 1.3930631627542231 K
Min x value of major axis: 224.39741027264316 K
Max x value of major axis: 225.7904734353974 K
Sample size: 1760800
[ 5.76101167e-01 -2.60594826e+02  2.95003959e+04]
224.39741027264316 225.7904734353974
IWP [g m-2] - lifecycle 3: 8.01719725488859


In [5]:
# Lifecycle 2 fitting
z2, minT2, maxT2 = qiT_fitting( 'ICON_0V2M0A0R_tst00000450_trim_extract_dt_filter.nc', 2000 )
print(z2)
print(minT2,maxT2)

# Lifecycle 2 IWP calculation
T_arr = np.linspace( minT2, maxT2, 100 )
qi_arr = z2[0]*T_arr**2 + z2[1]*T_arr + z2[2]
iwp2 = simps(qi_arr*10**(-6)/0.0055, T_arr ) # here in kg m-2

print('IWP [kg m-2] - lifecycle 2: ' + str(iwp2*10**3) )

Ellipse major axis: 2.9065018906637063 K
Min x value of major axis: 226.9188346259572 K
Max x value of major axis: 229.82533651662092 K
Sample size: 2754767
[5.55206397e-04 1.26835938e-01 2.89742905e+01]
226.9188346259572 229.82533651662092
IWP [kg m-2] - lifecycle 2: 45.92093197765202


/home/u18/sylvia/.conda/envs/ncplot/lib/python3.6/site-packages/ipykernel_launcher.py:2: RankWarning: Polyfit may be poorly conditioned
  


In [6]:
# Lifecycle 1 fitting
z1, minT1, maxT1 = qiT_fitting( 'ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc', 8000 )
print(z1)
print(minT1,maxT1)

# Lifecycle 1 IWP calculation
T_arr = np.linspace( minT1, maxT1, 100 )
qi_arr = z1[0]*T_arr**2 + z1[1]*T_arr + z1[2]
iwp1 = simps(qi_arr*10**(-6)/0.0055, T_arr ) # here in kg m-2

print('IWP [kg m-2] - lifecycle 1: ' + str(iwp1) )

Ellipse major axis: 2.295664827225788 K
Min x value of major axis: 226.49073264986367 K
Max x value of major axis: 228.78639747708945 K
Sample size: 6998897
[7.47207891e-05 1.70151167e-02 3.87451660e+00]
226.49073264986367 228.78639747708945
IWP [kg m-2] - lifecycle 1: 0.004850041214811204


/home/u18/sylvia/.conda/envs/ncplot/lib/python3.6/site-packages/ipykernel_launcher.py:2: RankWarning: Polyfit may be poorly conditioned
  


In [7]:
# Trajectory duration for both longwave and shortwave heating scalings
bd = '/xdisk/sylvia/traj_output/'
file = xr.open_dataset( bd + 'ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter.nc' ) #'_iwc'
T = file['T']
IWC = file['qi']

# Replace all your nan values with 10^(-8)
IWC = IWC.fillna( 10**(-8) )

# Find the arguments where qi > 10^(-8) and T < 237 K, i.e. non-negligible cirrus formation
IWC_masked = IWC.where( (IWC > 10**(-8)) & (T < 237), np.nan, 0 )

In [8]:
lifetimes = np.ndarray( (IWC.shape[1],) )
print(lifetimes.shape)
for i in np.arange(IWC.shape[1]):
    if i%100 == 0:
        print(i)
        print('Longest trajectory duration: ' + str((stop-start)*24/3600.) + ' hours')
    tr = IWC_masked[:,i] # Extract the trajectory of interest
    m = np.concatenate(( [True], np.isnan(tr), [True] ))  # Mask
    ss = np.flatnonzero(m[1:] != m[:-1]).reshape(-1,2)   # Start-stop limits
    if ss.size == 0:
        continue
    else:
        start, stop = ss[(ss[:,1] - ss[:,0]).argmax()]  # Get max interval, interval limits
        lifetimes[i] = stop-start

(19931,)
0


NameError: name 'stop' is not defined

In [10]:
lifetime1 = np.load( '/xdisk/sylvia/tropic_vis/output/ICON_0V1M0A0R_tst00001350_trim_extract_dt_filter_lifetimes.npy' )
lifetime2 = np.load( '/xdisk/sylvia/tropic_vis/output/ICON_0V2M0A0R_tst00000450_trim_extract_dt_filter_lifetimes.npy' )
lifetime3 = np.load( '/xdisk/sylvia/tropic_vis/output/CLAMS-Tf_0V2M0A0R_tst00000450_trim_extract_dt_iwc_filter_lifetimes.npy' )

# Converting here from timesteps to hours
print( np.nanmean(lifetime1)*24/3600 )
print( np.nanmean(lifetime2)*24/3600 )
print( np.nanmean(lifetime3)*24/3600 )

3.0088943521816933
8.287202904494979
3.648867520393747


In [17]:
# Longwave heating scalings
kappai = 100
HLW1 = (1-np.exp(-kappai*iwp1))*np.nanmean(lifetime1)
HLW2 = (1-np.exp(-kappai*iwp2))*np.nanmean(lifetime2)
HLW3 = (1-np.exp(-kappai*iwp3))*np.nanmean(lifetime3)
print( 'LW heating scaling 1 - ' + str(HLW1) )
print( 'LW heating scaling 2 - ' + str(HLW2) )
print( 'LW heating scaling 3 - ' + str(HLW3) )

HLW12 = (1-np.exp(-kappai*iwp1))*np.nanmean(lifetime1)/(1-np.exp(-kappai*iwp2))/np.nanmean(lifetime2)
print( 'LW heating ratio 1:2 - ' + str(HLW12) )

HLW13 = (1-np.exp(-kappai*iwp1))*np.nanmean(lifetime1)/(1-np.exp(-kappai*iwp3))/np.nanmean(lifetime3)
print('LW heating ratio 1:3 - ' + str(HLW13) )

HLW23 = (1-np.exp(-kappai*iwp2))*np.nanmean(lifetime2)/(1-np.exp(-kappai*iwp3))/np.nanmean(lifetime3)
print('LW heating ratio 2:3 - ' + str(HLW23) )

print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Not accumulated (no cloud lifetime included)')
HLW12 = (1-np.exp(-kappai*iwp1))/(1-np.exp(-kappai*iwp2))
print( 'LW heating ratio 1:2 - ' + str(HLW12) )

HLW13 = (1-np.exp(-kappai*iwp1))/(1-np.exp(-kappai*iwp3))
print('LW heating ratio 1:3 - ' + str(HLW13) )

HLW23 = (1-np.exp(-kappai*iwp2))/(1-np.exp(-kappai*iwp3))
print('LW heating ratio 2:3 - ' + str(HLW23) )

LW heating scaling 1 - 173.450125423663
LW heating scaling 2 - 1230.486006305909
LW heating scaling 3 - 301.82141942089083
LW heating ratio 1:2 - 0.1409606647574843
LW heating ratio 1:3 - 0.5746779859310989
LW heating ratio 2:3 - 4.076867734128547
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Not accumulated (no cloud lifetime included)
LW heating ratio 1:2 - 0.38823883249697655
LW heating ratio 1:3 - 0.6969084295129343
LW heating ratio 2:3 - 1.7950508068209832


In [14]:
b = [ 0.3159*10**(-4), 0.6289*10**(-4), 0.1237*10**(-6), -0.738*10**(-9) ]  # 1.242-1.299 [um]
b = [ 0.2693, 0.4137*10**(-2), -0.3817*10**(-4), 0.123*10**(-6) ]  # 3.077-3.846 [um]
#b = [ 0.2075, 0.8103*10**(-3), 0.5698*10**(-5), -0.4896*10**(-7) ] # 2.5-3.077 [um]
#b = [ -0.2841*10**(-3), 0.9704*10**(-3), -0.8236*10**(-7), -0.1237*10**(-7) ] # 2.15-2.5 [um]

c = [ 0.7717, 0.6579*10**(-3), -0.6515*10**(-6), -0.1075*10**(-7) ] # 1.242-1.299 [um]
c = [ 0.8011, 0.4177*10**(-2), -0.4192*10**(-4), 0.1422*10**(-6) ] # 3.077-3.846 [um]
#c = [ 0.8837, 0.1185*10**(-2), -0.585*10**(-5), 0.3932*10**(-8) ] # 2.5-3.077 [um]
#c = [ 0.8172, 0.1637*10**(-3), 0.1079*10**(-4), -0.6696*10**(-7) ] # 2.15-2.5 [um]

De = 14.63
omega1 = 1 - (b[0] + b[1]*De + b[2]*De**2 + b[3]*De**3)
g1 = c[0] + c[1]*De + c[2]*De**2 + c[3]*De**3

De = 5.03
omega2 = 1 - (b[0] + b[1]*De + b[2]*De**2 + b[3]*De**3)
g2 = c[0] + c[1]*De + c[2]*De**2 + c[3]*De**3

De = 9.18
omega3 = 1 - (b[0] + b[1]*De + b[2]*De**2 + b[3]*De**3)
g3 = c[0] + c[1]*De + c[2]*De**2 + c[3]*De**3

print(omega1,omega2,omega3)
print(g1,g2,g3)
print(kappai*iwp1,kappai*iwp2,kappai*iwp3)

0.677960321211819 0.7108409719391791 0.695843862170264
0.8536823625222434 0.8210677931455393 0.8360221698458704
0.4850041214811204 4.592093197765202 0.801719725488859


In [15]:
# Shortwave cooling scalings
# Optical depth = kappai*iwp

HSW1 = (omega1*(1-g1)*kappai*iwp1*np.nanmean(lifetime1))
HSW2 = (omega2*(1-g2)*kappai*iwp2*np.nanmean(lifetime2))
HSW3 = (omega3*(1-g3)*kappai*iwp3*np.nanmean(lifetime3))
print( 'SW heating scaling 1 - ' + str(HSW1) )
print( 'SW heating scaling 2 - ' + str(HSW2) )
print( 'SW heating scaling 3 - ' + str(HSW3) )

HSW12 = (omega1*(1-g1)*kappai*iwp1*np.nanmean(lifetime1))/(omega2*(1-g2)*kappai*iwp2*np.nanmean(lifetime2))
print('SW heating ratio 1:2 - ' + str(HSW12) )

HSW13 = (omega1*(1-g1)*kappai*iwp1*np.nanmean(lifetime1))/(omega3*(1-g3)*kappai*iwp3*np.nanmean(lifetime3))
print('SW heating ratio 1:3 - ' + str(HSW13) )

HSW23 = (omega2*(1-g2)*kappai*iwp2*np.nanmean(lifetime2))/(omega3*(1-g3)*kappai*iwp3*np.nanmean(lifetime3))
print('SW heating ratio 2:3 - ' + str(HSW23) )

SW heating scaling 1 - 21.714237534833135
SW heating scaling 2 - 726.0572982639532
SW heating scaling 3 - 50.06899335781854
SW heating ratio 1:2 - 0.029907057730503072
SW heating ratio 1:3 - 0.433686321185891
SW heating ratio 2:3 - 14.50113632353616
